# SpiralSafe Git & VCS Insights

This notebook provides interactive analysis of the repository's version control status, including branch health, commit patterns, and contributor stats.

In [ ]:
import subprocess
import pandas as pd
import re
from datetime import datetime, timedelta
from IPython.display import display, Markdown, HTML

# 🎨 GITHUB FLAIR CSS
GITHUB_CSS = """
<style>
:root {
    --gh-bg: #0d1117;
    --gh-card-bg: #161b22;
    --gh-border: #30363d;
    --gh-text: #c9d1d9;
    --gh-text-muted: #8b949e;
    --gh-link: #58a6ff;
    --gh-btn: #238636;
    --gh-danger: #f85149;
    --gh-success: #3fb950;
    --gh-warning: #d29922;
    --gh-header-bg: #161b22;
}

div.gh-container {
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Helvetica, Arial, sans-serif;
    color: var(--gh-text);
    background-color: var(--gh-bg);
    padding: 20px;
    border-radius: 6px;
    border: 1px solid var(--gh-border);
}

div.gh-card {
    background-color: var(--gh-card-bg);
    border: 1px solid var(--gh-border);
    border-radius: 6px;
    padding: 16px;
    margin-bottom: 16px;
}

div.gh-header {
    background-color: var(--gh-header-bg);
    border-bottom: 1px solid var(--gh-border);
    padding: 16px;
    margin: -16px -16px 16px -16px;
    border-top-left-radius: 6px;
    border-top-right-radius: 6px;
    font-weight: 600;
}

.gh-stat-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
    gap: 16px;
}

.gh-stat-box {
    text-align: center;
    padding: 10px;
}

.gh-stat-value {
    font-size: 24px;
    font-weight: 600;
    color: var(--gh-text);
}

.gh-stat-label {
    font-size: 12px;
    color: var(--gh-text-muted);
}

table.gh-table {
    width: 100%;
    border-collapse: collapse;
}

table.gh-table th {
    text-align: left;
    padding: 8px 16px;
    color: var(--gh-text-muted);
    font-weight: 600;
    border-bottom: 1px solid var(--gh-border);
}

table.gh-table td {
    padding: 8px 16px;
    border-bottom: 1px solid var(--gh-border);
    color: var(--gh-text);
}

.gh-badge {
    display: inline-block;
    padding: 2px 7px;
    font-size: 12px;
    font-weight: 500;
    line-height: 18px;
    border-radius: 2em;
    border: 1px solid transparent;
}

.gh-badge-success { color: #3fb950; border-color: #3fb950; }
.gh-badge-warning { color: #d29922; border-color: #d29922; }
.gh-badge-danger { color: #f85149; border-color: #f85149; }
.gh-badge-neutral { color: #8b949e; border-color: #8b949e; }

.gh-branch {
    font-family: ui-monospace, SFMono-Regular, SF Mono, Menlo, Consolas, Liberation Mono, monospace;
    background-color: rgba(110,118,129,0.4);
    padding: 2px 4px;
    border-radius: 4px;
    font-size: 85%;
    color: #58a6ff;
}
</style>
"""

display(HTML(GITHUB_CSS))

def run_git(command):
    try:
        # Run from repo root
        result = subprocess.run(f"cd .. && {command}", shell=True, capture_output=True, text=True)
        return result.stdout.strip()
    except Exception as e:
        return ""

def badge(text, type="neutral"):
    return f'<span class="gh-badge gh-badge-{type}">{text}</span>'

print("✅ GitHub Insights System Loaded")


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# 📊 REPOSITORY DASHBOARD & BRANCH HEALTH

# Get Data
repo_name = "SpiralSafe" # Hardcoded or parsing .git/config
current_branch = run_git("git rev-parse --abbrev-ref HEAD")
total_commits = run_git("git rev-list --count HEAD")
last_commit_date = run_git("git log -1 --format=%cd --date=relative")

# Branch Data
raw_branches = run_git("git for-each-ref --sort=-committerdate --format='%(committerdate:short)|%(refname:short)|%(authorname)' refs/heads/")
branches_html = ""

if raw_branches:
    lines = [line.split('|') for line in raw_branches.split('\n') if line]
    
    for date, branch, author in lines:
        try:
            days_ago = (datetime.now() - datetime.strptime(date, '%Y-%m-%d')).days
        except:
            days_ago = 0
            
        status_cls = "gh-badge-success" if days_ago < 3 else ("gh-badge-warning" if days_ago < 14 else "gh-badge-neutral")
        status_text = "Active" if days_ago < 3 else ("Stale" if days_ago < 45 else "Dormant")
        
        # Categorize
        if branch == 'main': cat = "Production"
        elif 'integration/' in branch: cat = "Integration"
        elif 'feat/' in branch: cat = "Feature"
        elif 'fix/' in branch: cat = "Fix"
        elif 'ops/' in branch: cat = "Ops"
        else: cat = "Other"
        
        branches_html += f"""
        <tr>
            <td><span class="gh-badge gh-badge-neutral">{cat}</span></td>
            <td><span class="gh-branch">{branch}</span></td>
            <td>{author}</td>
            <td>{date}</td>
            <td><span class="gh-badge {status_cls}">{status_text}</span></td>
        </tr>
        """

dashboard_html = f"""
<div class="gh-container">
    <div class="gh-card">
        <div class="gh-header">Repository Overview</div>
        <div class="gh-stat-grid">
            <div class="gh-stat-box">
                <div class="gh-stat-value">{repo_name}</div>
                <div class="gh-stat-label">Repository</div>
            </div>
            <div class="gh-stat-box">
                <div class="gh-stat-value"><span class="gh-branch">{current_branch}</span></div>
                <div class="gh-stat-label">Current Branch</div>
            </div>
            <div class="gh-stat-box">
                <div class="gh-stat-value">{total_commits}</div>
                <div class="gh-stat-label">Total Commits</div>
            </div>
             <div class="gh-stat-box">
                <div class="gh-stat-value">{len(lines) if raw_branches else 0}</div>
                <div class="gh-stat-label">Branches</div>
            </div>
        </div>
    </div>

    <div class="gh-card">
        <div class="gh-header">🌿 Branch Health</div>
        <table class="gh-table">
            <thead>
                <tr>
                    <th>Type</th>
                    <th>Branch</th>
                    <th>Last Author</th>
                    <th>Last Active</th>
                    <th>Status</th>
                </tr>
            </thead>
            <tbody>
                {branches_html}
            </tbody>
        </table>
    </div>
</div>
"""

display(HTML(dashboard_html))


## Recent Commits

In [ ]:
# 1. Check for "Initial plan" commits (Copilot noise)
plan_commits = run_git("git log --grep='Initial plan' --oneline")
plan_count = len(plan_commits.split('\n')) if plan_commits else 0

# 2. Check for Desktop paths in code (Privacy/Reproducibility)
desktop_grep = run_git("grep -r 'Desktop' . --include=*.{py,ps1,sh,md} --exclude-dir=books")
desktop_count = len(desktop_grep.split('\n')) if desktop_grep else 0

# 3. Check for "H&&S" markers in recent commits
hs_commits = run_git("git log -n 50 --grep='H&&S' --oneline")
hs_count = len(hs_commits.split('\n')) if hs_commits else 0

# Display Dashboard
display(Markdown("### 🚨 Pattern Violations"))

if plan_count > 0:
    display(Markdown(f"**❌ 'Initial plan' Commits Found:** {plan_count} (Action: Squash these)"))
else:
    display(Markdown(f"✅ No 'Initial plan' noise found."))

if desktop_count > 0:
    display(Markdown(f"**warning Desktop Paths Detected:** {desktop_count} occurrences (Action: Use relative paths)"))
else:
    display(Markdown(f"✅ No local Desktop paths detected."))

display(Markdown("---"))
display(Markdown("### 📡 Protocol Adherence"))
display(Markdown(f"**H&&S Markers (Last 50 commits):** {hs_count}"))
if hs_count < 5:
    display(Markdown("*Tip: Use `H&&S:WAVE` or `H&&S:SYNC` in commit messages to track handoffs.*"))

## 🛡️ Anti-Pattern Monitor
Checking for `GIT_INSIGHTS_ANALYSIS.md` violations.

In [ ]:
log = run_git("git log -n 50 --pretty=format:'%h|%ad|%an|%s' --date=short")
if log:
    data = [line.split('|') for line in log.split('\n') if len(line.split('|')) >= 4]
    df_log = pd.DataFrame(data, columns=['Hash', 'Date', 'Author', 'Message'])

    # Analyze Contributors
    author_counts = df_log['Author'].value_counts()
    
    display(Markdown("### 👥 Collaboration Mix (Last 50 Commits)"))
    display(author_counts.to_frame(name="Commits").T)
    
    display(Markdown("### 📜 Recent History"))
    display(df_log.head(10).style.hide(axis='index'))
else:
    print("Could not retrieve git log.")

In [ ]:
display(Markdown("---"))
display(Markdown(f"**{badge('SpiralSafe System', '#663399')}** • *Repository Analysis Complete*"))
display(Markdown("*Run `ops/scripts/spiralsafe wave analyze` for full coherence validation.*"))